In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 3, Finished, Available, Finished)

###### **<mark>Creating dimension table of Order_Priority</mark>**

In [2]:
%%sql
create table if not exists Ecommerce_project.Gold_OrderPriority
(
    Order_Priority string,
    Priority_ID int,
    Created_TS TIMESTAMP,
    Modified_TS TIMESTAMP
)
USING DELTA 

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

###### **<mark>Building increamental load</mark>**

In [3]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Ecommerce_project.gold_orderpriority").first()[0]

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 5, Finished, Available, Finished)

###### **<mark>Creating dataframe with required columns for gold Order_priority table</mark>**

In [4]:
df_bronze=spark.sql("select distinct Order_Priority from Ecommerce_project.bronze_sales where Modified_TS>'{}'".format(Max_Date))

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 6, Finished, Available, Finished)

###### **<mark>Creating priority_ID columns using montinically_increasing_id</mark>**

In [5]:
Max_ID = spark.sql("select coalesce(max(Priority_ID),0) from Ecommerce_project.gold_orderpriority").first()[0]

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 7, Finished, Available, Finished)

In [6]:
final_df= df_bronze.withColumn('Priority_ID',monotonically_increasing_id()+Max_ID+1)

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 8, Finished, Available, Finished)

###### **<mark>Creating temporary view from df we just created</mark>**

In [7]:
final_df.createOrReplaceTempView("ViewOrderPriority")

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 9, Finished, Available, Finished)

###### **<mark>Inserting data from view to gold_table</mark>**

In [8]:
%%sql
MERGE INTO Ecommerce_project.gold_orderpriority as gop
USING ViewOrderPriority as vop
on gop.Order_Priority = vop.Order_Priority
WHEN MATCHED THEN

UPDATE SET
gop.Modified_TS = CURRENT_TIMESTAMP()

WHEN NOT MATCHED THEN
INSERT
(
gop.Priority_ID,
gop.Order_Priority,
gop.Created_TS,
gop.Modified_TS

)
VALUES
(
vop.Priority_ID,
vop.Order_Priority,
CURRENT_TIMESTAMP(),
CURRENT_TIMESTAMP()
)

StatementMeta(, ad884fd4-6fae-405b-a217-2c38a498c4ec, 10, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>